In [1]:
import pandas as pd
import numpy as np
import os

import emoji

import matplotlib.pyplot as plt

import urllib.request

from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

from nltk.classify import NaiveBayesClassifier
from nltk.corpus import subjectivity
from nltk.sentiment import SentimentAnalyzer
from nltk.sentiment.util import *

In [4]:
from konlpy.tag import Hannanum
hannanum = Hannanum()

In [5]:
print(hannanum.analyze('질문이나 건의사항은 깃헙 이슈 트래커에 남겨주세요.'))

[[[('질문', 'ncpa'), ('이나', 'jcj')], [('질문', 'ncpa'), ('이나', 'jxc')], [('질문', 'ncpa'), ('이', 'jcc'), ('나', 'jxc')], [('질문', 'ncpa'), ('이', 'jcs'), ('나', 'jxc')], [('질문', 'ncpa'), ('이', 'jp'), ('나', 'ef')], [('질문', 'ncpa'), ('이', 'ncn'), ('나', 'jcj')], [('질문', 'ncpa'), ('이', 'ncn'), ('나', 'jxc')], [('질문', 'ncpa'), ('이', 'ncn'), ('이', 'jp'), ('나', 'ef')], [('질문', 'ncpa'), ('이', 'ncn'), ('이', 'jp'), ('나', 'ecc')], [('질문', 'ncpa'), ('이', 'ncn'), ('이', 'jp'), ('나', 'ecs')], [('질문', 'ncpa'), ('이', 'jp'), ('나', 'ecc')], [('질문', 'ncpa'), ('이', 'jp'), ('나', 'ecs')]], [[('건의', 'ncpa'), ('사항', 'ncn'), ('은', 'jxc')], [('건의', 'ncpa'), ('사항', 'ncn'), ('은', 'ncn')]], [[('깃헙', 'ncn')], [('깃헙', 'nqq')]], [[('이슈', 'ncn')]], [[('트래커', 'ncn'), ('에', 'jca')], [('트래커에', 'ncn')], [('트래커', 'nqq'), ('에', 'jca')], [('트래커에', 'nqq')]], [[('남기', 'pvg'), ('어', 'ecx'), ('주', 'px'), ('세요', 'ef')], [('남기', 'pvg'), ('어', 'ecx'), ('주', 'px'), ('세', 'ef'), ('요', 'jxf')], [('남기', 'pvg'), ('어', 'ecx'), ('줄', 'px'), ('세요', 'e

In [6]:
print(hannanum.morphs('질문이나 건의사항은 깃헙 이슈 트래커에 남겨주세요.'))

['질문', '이나', '건의사항', '은', '깃헙', '이슈', '트래커', '에', '남기', '어', '주', '세', '요', '.']


In [2]:
import konlpy

from konlpy.tag import Kkma
from konlpy.utils import pprint
kkma = Kkma()

In [3]:
print(kkma.nouns('질문이나 건의사항은 깃헙 이슈 트래커에 남겨주세요.'))

['질문', '건의', '건의사항', '사항', '깃헙', '이슈', '트래커']


In [9]:
file = r"/Users/reejungkim/Documents/Git/WebCrawler/GooglePlayStore_Review_2021-02-14-17:02:09.csv"

In [10]:
df = pd.read_csv(file)

In [11]:
df.head()

,Unnamed: 0,author,developer_reply,review,review_date,reviewer_ratings
0,0,임닉이,NaN,회원가입을하려니까 가입 필수조건을 다 체크하고 전화번호 인증하니까 필수조건 체크가...,2021년 2월 8일,52
1,1,Nayeon kang,NaN,나름 대기업인데 일좀 열심히 했으면 좋겠다. 쇼핑앱 여러개 사용중인데 이렇게 뭐 ...,2021년 2월 5일,51
2,2,산뜻산뜻,NaN,주문조회에서 택배&12개월로 보고 싶은데 택배배송을 누르면 12개월이 3개월로 바...,2021년 1월 25일,51
3,3,신종철,NaN,시스템 잘도 만들어놨네요.왠만해서 귀찮아서 글도 잘 안쓰는 사람인데요. 시스템 왜...,2021년 1월 20일,51
4,4,김재규,"<div class=""LVQB0b""><div class=""QoPmEb""></div>...","장바구니에 있는 목록들의 순서를 이동, 변경할 수 있게 해주면 좋겠습니다. 장바구...",2020년 12월 15일,55


In [12]:
df.shape

(200, 6)

In [13]:
def language_preprocessor(string):
    text =  re.sub('[^a-zA-Z]', ' ', string) #replace non-english with blank
    text = text.lower().split() #decapitalize texts
    text = [word for word in text if not word in stops] #remove stopword
    text = [lm.lemmatize(word) for word in text]  #lemmatization of texts
    return text #' '.join(text)
    

In [14]:
import re

def clean_text(texts):
    corpus = []
    for i in range(0, len(texts)):
        review = re.sub(r'[@%\\*=()/~#&\+á?\xc3\xa1\-\|\.\:\;\!\-\,\_\~\$\'\"]', '',str(texts[i])) #remove punctuation
        review = re.sub(r'\d+','', str(texts[i]))# remove number
        review = review.lower() #lower case
        review = re.sub(r'\s+', ' ', review) #remove extra space
        review = re.sub(r'<[^>]+>','',review) #remove Html tags
        review = re.sub(r'\s+', ' ', review) #remove spaces
        review = re.sub(r"^\s+", '', review) #remove space from start
        review = re.sub(r'\s+$', '', review) #remove space from the end
        corpus.append(review)
    return corpus

df['text processed'] = df['review'].apply(lambda x: clean_text(x)) 

In [15]:
df['nouns'] = df['review'].apply(lambda x: kkma.nouns(x))

In [16]:
df.head()

,Unnamed: 0,author,developer_reply,review,review_date,reviewer_ratings,nouns
0,0,임닉이,NaN,회원가입을하려니까 가입 필수조건을 다 체크하고 전화번호 인증하니까 필수조건 체크가...,2021년 2월 8일,52,"[회원, 가입, 하려, 필수, 필수조건, 조건, 체크, 전화, 전화번호, 번호, 인..."
1,1,Nayeon kang,NaN,나름 대기업인데 일좀 열심히 했으면 좋겠다. 쇼핑앱 여러개 사용중인데 이렇게 뭐 ...,2021년 2월 5일,51,"[나름, 대기업, 일, 쇼핑, 쇼핑앱, 앱, 개, 사용, 사용중, 중, 뭐, 이벤트..."
2,2,산뜻산뜻,NaN,주문조회에서 택배&12개월로 보고 싶은데 택배배송을 누르면 12개월이 3개월로 바...,2021년 1월 25일,51,"[주문, 주문조회, 조회, 택배, 12, 12개월, 개월, 택배배송, 배송, 3, ..."
3,3,신종철,NaN,시스템 잘도 만들어놨네요.왠만해서 귀찮아서 글도 잘 안쓰는 사람인데요. 시스템 왜...,2021년 1월 20일,51,"[시스템, 글, 사람, 식, 이해, 상담, 상담사, 사, 기본, 기본적인, 적인, ..."
4,4,김재규,"<div class=""LVQB0b""><div class=""QoPmEb""></div>...","장바구니에 있는 목록들의 순서를 이동, 변경할 수 있게 해주면 좋겠습니다. 장바구...",2020년 12월 15일,55,"[장, 장바구니, 바구니, 목록, 순서, 이동, 변경, 수, 시간, 물품, 아래, ..."


In [18]:
df['nouns'] = df['nouns'].apply(lambda x: clean_text(x))

In [19]:
df.head()

,Unnamed: 0,author,developer_reply,review,review_date,reviewer_ratings,nouns
0,0,임닉이,NaN,회원가입을하려니까 가입 필수조건을 다 체크하고 전화번호 인증하니까 필수조건 체크가...,2021년 2월 8일,52,"[회원, 가입, 하려, 필수, 필수조건, 조건, 체크, 전화, 전화번호, 번호, 인..."
1,1,Nayeon kang,NaN,나름 대기업인데 일좀 열심히 했으면 좋겠다. 쇼핑앱 여러개 사용중인데 이렇게 뭐 ...,2021년 2월 5일,51,"[나름, 대기업, 일, 쇼핑, 쇼핑앱, 앱, 개, 사용, 사용중, 중, 뭐, 이벤트..."
2,2,산뜻산뜻,NaN,주문조회에서 택배&12개월로 보고 싶은데 택배배송을 누르면 12개월이 3개월로 바...,2021년 1월 25일,51,"[주문, 주문조회, 조회, 택배, , 개월, 개월, 택배배송, 배송, , 개월, 전..."
3,3,신종철,NaN,시스템 잘도 만들어놨네요.왠만해서 귀찮아서 글도 잘 안쓰는 사람인데요. 시스템 왜...,2021년 1월 20일,51,"[시스템, 글, 사람, 식, 이해, 상담, 상담사, 사, 기본, 기본적인, 적인, ..."
4,4,김재규,"<div class=""LVQB0b""><div class=""QoPmEb""></div>...","장바구니에 있는 목록들의 순서를 이동, 변경할 수 있게 해주면 좋겠습니다. 장바구...",2020년 12월 15일,55,"[장, 장바구니, 바구니, 목록, 순서, 이동, 변경, 수, 시간, 물품, 아래, ..."
